In [ ]:
from __future__ import print_function, division
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

from itertools import chain
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from IPython.core.debugger import set_trace
import itertools
import seaborn as sns
from tqdm import tqdm
import random
import cv2

from natsort import natsorted
import collections
from IPython import display
import pylab as pl

from torch import nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF

from skorch import NeuralNetRegressor
from skorch.helper import predefined_split
from skorch import callbacks
from sklearn.model_selection import GridSearchCV
from collections import defaultdict         

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics.regression import median_absolute_error, mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.model_selection import KFold
import c3d_wrapper
from data_utils import *
import models
from params import *

# TF model for feature extraction

In [ ]:
# pretrained c3d net( tensorflow )
# tf_model = TF_Model()

#visualize_conv_featsmap('7157030_test_0_trial_1', tf_model, layer='conv1')

In [ ]:
from skorch.callbacks import Callback
from torchvision.utils import save_image

def to_tensor_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.permute(0,3,1,2)
    x = x.view(x.size(0), 1, 64, 64)
    return x

class Save_Reconstruction_Results(Callback):
    def __init__(self, path):
        self.path = path
        
    def on_epoch_end(self, net, **kwargs):
        for name in ['train', 'valid']:
            dataset = kwargs['dataset_'+name]
            rand_ix = np.random.randint(len(dataset))
            X,y = dataset[rand_ix]
            
            save_dir = os.path.join(self.path, name)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            
            # target img
            y = y.numpy().transpose(1,2,3,0)  # (maxlen,h,w,3)
            
            # predicted img
            pred = net.predict(X[None,:])[0].transpose(1,2,3,0) # (maxlen,h,w,3)
            
            for sub_name,pic in zip(['target', 'pred'], [y,pred]):
                pic = to_tensor_img(torch.from_numpy(pic))
                save_image(pic, os.path.join(save_dir,sub_name+'.png'))

                
def fetch_samples_from_dataset(dataset):
    X = []
    Y = []
    for item in dataset:
        X.append(item[0].numpy())
        Y.append(item[1].numpy())
        
    return np.array(X), np.array(Y)

def check_array(arr):
    dim = arr.shape[1]
    
    # select non-nan rows
    try:
        arr = arr[np.where(~np.isnan(arr))].reshape(-1, dim)
    except:
        set_trace()
        
    # select non-inf rows
    arr = arr[(arr!=np.inf).all(1)]
    
    return arr

def mape(y_true, y_pred, scaler=None, reduce_all=True):
    if scaler:
        # when skorch callbacks is executed for epoch_scoring
        y_pred = scaler.inverse_transform(y_pred)
        y_true = scaler.inverse_transform(y_true)
        
    with np.errstate(divide='ignore'):
        mape = np.abs((y_true-y_pred)/y_true)

    if reduce_all:
        return check_array(mape).mean()
    else:
        return check_array(mape).mean(0)

def root_mean_squared_error(y_true, y_pred, reduce_all=True):
    if reduce_all:
        return np.sqrt(mean_squared_error(y_true, y_pred))
    else:
        return np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values'))

def record_score(y_pred, y_true, epoch, phase, log_path="score_log.txt"):
    MAE = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
    MAPE = mape(y_true, y_pred, reduce_all=False)
    RMSE = root_mean_squared_error(y_true, y_pred, reduce_all=False)
    R2 = r2_score(y_true, y_pred, multioutput='raw_values')
    EV = explained_variance_score(y_true, y_pred, multioutput='raw_values')
    
    deco_ = f"=== EPOCH-{epoch} / {phase} === summary"
    mae_ = f"MAE : {MAE}"
    mape_ = f"MAPE : {MAPE}"
    rmse_ = f"RMSE : {RMSE}"
    r2_ = f"R^2 : {R2}"
    ev_ = f"Explained variation : {EV}"
    
    msg = '\n'.join([deco_, mae_, mape_, rmse_, r2_, ev_])
    os.system("echo \'{}\' >> {}".format(msg, log_path))
    
    print(f'resulting file is saved to \"{log_path}\"')

class Report_Regression_Results(Callback):
    def __init__(self, columns, scores, scaler=None, period=5, log_path="score_log.txt"):
        self.columns = columns
        self.scores = scores
        self.scaler = scaler
        self.period = period
        
        self.loss_history = defaultdict(list)
        if os.path.exists(log_path):
            os.system(f"rm {log_path}")
            
        self.log_path = log_path
        self.epoch_scores = { k:[] for k in scores }
        
    def on_epoch_end(self, net, **kwargs):
        if len(net.history) % self.period == 0:
            for phase in ['train', 'valid']:
                self.loss_history[phase].append(net.history[-1, f'{phase}_loss'])
            
            for phase in ['train', 'valid']:
                dataset = kwargs['dataset_'+phase]

                X, y_true = fetch_samples_from_dataset(dataset)
                y_pred = net.predict(X)
                if self.scaler:
                    y_pred = self.scaler.inverse_transform(y_pred)
                    y_true = self.scaler.inverse_transform(y_true)
                
                record_score(y_pred, y_true, epoch_scores=self.epoch_scores,
                             epoch=len(net.history), phase=phase, log_path=self.log_path)

                report_lerning_process(columns=self.columns,
                                       epoch=len(net.history),
                                       phase=phase,
                                       y_pred=y_pred,
                                       y_true=y_true,
                                       loss_history=self.loss_history)

# Custom criterion

In [ ]:
from torch.nn.modules.loss import _Loss

class MyCriterion(_Loss):
    def __init__(self):
        super(MyCriterion, self).__init__()
        
    def forward(self, x, y):
        set_trace()
        return nn.MSELoss()(x,y)
    
#         set_trace()
        
#         valid_mask = ~(y.view(y.size(0),FRAME_MAXLEN,-1)==0).all(dim=2)
#         valid_mask = valid_mask.float()
#         return torch.mean(torch.sum((valid_mask * ((x-y)**2).mean((1,3,4))),1)/torch.sum(valid_mask,1))

# Setup for distributed computing (dask)

In [ ]:
from dask.distributed import Client
client = Client('127.0.0.1:8786')
from sklearn.externals import joblib

client

# Grid search (or random search)

In [ ]:
from dask_searchcv import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer

In [ ]:
def grid_search(model, super_class,
                task = 'regression@pretrained',
                n_splits=2,
                feature_extraction_model=None, feature_layer='conv1',
                # dataset path
                input_file = "../../preprocess/data/person_detection_and_tracking_results_drop.pkl",
                target_file = "../../preprocess/data/targets_dataframe.pkl"):
    

    data_dict = prepare_dataset(input_file, target_file,
                                feature_extraction_model=feature_extraction_model, layer=feature_layer)

    
    train_dataset = dataset_init(task, 
                                data_dict['train_X'], data_dict['train_y'],
                                scaler=data_dict['scaler'], name='train')
    
    # holdouf test set for final evaluation
    test_dataset = dataset_init(task, 
                                data_dict['test_X'], data_dict['test_y'],
                                scaler=data_dict['scaler'], name='test')
    
    # init net
    net = super_class(
        model,
        batch_size=8,
        max_epochs=1,
        optimizer=torch.optim.SGD,
        device='cuda',
        # Shuffle training data on each epoch
        iterator_train__shuffle=True,
        callbacks=[('ealy_stop', callbacks.EarlyStopping()),
                   ('lr_scheduler', callbacks.LRScheduler(policy='CyclicLR',
                                                         mode='exp_range',
                                                         base_lr=1e-3,
                                                         gamma=0.97)),
                   ('MAPE', callbacks.EpochScoring(scoring=make_scorer(mape, 
                                                                       scaler = data_dict['scaler']), lower_is_better=True)),
                   ('R2', callbacks.EpochScoring(scoring='r2', lower_is_better=False)),]
    )
    
#     params = {'callbacks__lr_scheduler__base_lr': [1e-3, 1e-4],
#               'optimizer__weight_decay': [1.0, 1e-1, 1e-2, 1e-3]},
    params = {'callbacks__lr_scheduler__base_lr': [1e-3,],
              'optimizer__weight_decay': [1e-1,]},

    gs = GridSearchCV(net, params, refit=True, cv=n_splits, scoring='r2')

    X, y_true = fetch_samples_from_dataset(train_dataset)
    
    gs.fit(X, y_true)
    
    set_trace()
    
    print(gs.best_estimator_, gs.best_score_, gs.best_params_)
    
    
    return gs.best_estimator_, gs.best_score_, gs.best_params_, \
            train_dataset, test_dataset, data_dict['scaler']

task = 'regression@pretrained'

super_class = NeuralNetRegressor

if task.split('@')[1]=='fromscratch':
    super_class = AutoEncoderNet

net, best_score, best_params, train_dataset, test_dataset, scaler = \
        grid_search(
                    model=eval('_'.join(task.split('@')).capitalize()),
                    super_class=super_class,
                    #feature_extraction_model=tf_model, feature_layer='fc1',
                    task=task
        )

# Performance Evaluation for Testset

In [ ]:
X, y_true = fetch_samples_from_dataset(test_dataset)
y_pred = net.predict(X)

if scaler:
    y_pred = scaler.inverse_transform(y_pred)
    y_true = scaler.inverse_transform(y_true)


record_score(y_pred, y_true, epoch=10, phase='test', log_path='./score_log.txt')

report_lerning_process(columns=target_columns,
                       phase='test',
                       y_pred=y_pred,
                       y_true=y_true)

# Cross validation (base_lr = 1e-3, weight_decay = 1e-1)

In [ ]:
def cross_validation_loop(n_splits=5,
                          feature_extraction_model=None, feature_layer='conv1',
                          # dataset path
                          input_file = "../../preprocess/data/person_detection_and_tracking_results_drop.pkl",
                          target_file = "../../preprocess/data/targets_dataframe.pkl",
                          callback_list=['early_stop', 'prog_bar', 'report_regression_results'],
                          scores=['MAE', 'MAPE', 'RMSE', 'R2', 'EV']):
    

    data_dict = prepare_dataset(input_file, target_file,
                                feature_extraction_model=feature_extraction_model, layer=feature_layer)

    # holdouf test set for final evaluation
    test_dataset = dataset_init(task, 
                                data_dict['test_X'], data_dict['test_y'],
                                scaler=data_dict['scaler'], name='test')
    
    # can be modified !
    CV_scores = { k:[] for k in scores }
    
    data_locations = np.array(data_dict['train_vids'])

    input_df = data_dict['input_df']
    target_df = data_dict['target_df']
    scaler = data_dict['scaler']
    
    if callback_list:
        callback_map = {'early_stop': callbacks.EarlyStopping(),
                        'prog_bar': callbacks.ProgressBar(),
                        'lr_scheduler': callbacks.LRScheduler(policy='CyclicLR',
                                                                mode='exp_range',
                                                                base_lr=1e-3,
                                                                gamma=0.97,
                                                               ),
                        'report_regression_results': Report_Regression_Results(columns=target_columns, 
                                                                                 period=1,
                                                                                 scaler=scaler,
                                                                                 scores=scores),
                        'save_reconstruction_results': Save_Reconstruction_Results(path='./results')
                         }

        callback_list = [ callback_map.get(cb) for cb in callback_list ]
    
    # K-fold CV
    kf = KFold(n_splits=n_splits)
    print('start!')
    
    for train, valid in kf.split(data_locations):
        
        # split trainset with train/valid
        train_split, valid_split = data_locations[train], data_locations[valid]

        train_X, train_y = filter_input_df_with_vids(input_df,train_split), filter_target_df_with_vids(target_df,train_split)
        valid_X, valid_y = filter_input_df_with_vids(input_df,valid_split), filter_target_df_with_vids(target_df,valid_split)

        # dsataset !!
        train_dataset = dataset_init(task, train_X, train_y, scaler=scaler, name='train')
        valid_dataset = dataset_init(task, valid_X, valid_y, scaler=scaler, name='valid')

        # init net
        net = super_class(
            model,
            batch_size=8,
            max_epochs=20,
            lr = 1e-3,
            optimizer=torch.optim.SGD,
            optimizer__weight_decay=1e-1,
            device='cuda',
            #criterion=MyCriterion,
            #criterion__mu=train_y.values.mean(0),
            #criterion__sigma=train_y.values.std(0),
            train_split=predefined_split(valid_dataset),
            # Shuffle training data on each epoch
            iterator_train__shuffle=True,
            warm_start=False, # re-init the module
            callbacks=callback_list,
        )
        
        # implicit train/validate loop for each CV split
        net.fit(train_dataset, y=None)
        
        epoch_scores = callback_map['report_regression_results'].epoch_scores
        for k,v in epoch_scores.items():
            CV_scores[k].append(v)

    for k,v in CV_scores.items():
        CV_scores[k] = np.array(v).mean(0)   # avg for all CV split
        
    set_trace()
    
    return net, train_dataset, valid_dataset, test_dataset, CV_scores, scaler

task = 'regression@pretrained'
# task = 'reconstruction@pretrained'
# task = 'reconstruction@fromscratch'

super_class = NeuralNetRegressor
if task.split('@')[1]=='fromscratch':
    super_class = AutoEncoderNet

net, train_dataset, valid_dataset, test_dataset, CV_scores, scaler = cross_validation_loop(
    model=models_fuck.Regression_pretrained,
    super_class=super_class,
    #feature_extraction_model=tf_model, feature_layer='fc1',

    task=task,
    callback_list=['early_stop',
                 'lr_scheduler',
                 'prog_bar', 'report_regression_results'])

In [ ]:
X, y_true = fetch_samples_from_dataset(valid_dataset)
y_pred = net.predict(X)

# Check normality of target data

In [ ]:
from statsmodels.graphics.gofplots import qqplot
qqplot(y_true[:,0], line='s'); plt.show()

In [ ]:
for ix,col in enumerate(target_columns):
    sampled_y_true = scaler.inverse_transform(y_true)[:,ix] if scaler else y_true[:,ix]
    sampled_y_pred = scaler.inverse_transform(y_pred)[:,ix] if scaler else y_pred[:,ix]

    sorted_ixs = np.argsort(sampled_y_true)

    sampled_y_true = sampled_y_true[sorted_ixs]
    sampled_y_pred = sampled_y_pred[sorted_ixs]
    

    fig, axes = plt.subplots(2, figsize=(11,15))
    axes[0].set_title(col+' trace')
    axes[0].plot(sampled_y_true, 'g*')
    axes[0].plot(sampled_y_pred)
    axes[0].axhline(y=sampled_y_true.mean(), color='r', linestyle='--')
    axes[1].set_title(col+' residuals')
    axes[1].scatter(sampled_y_pred, sampled_y_pred-sampled_y_true)
    axes[1].axhline(y=0.0, color='r', linestyle='--')
    plt.savefig(f'figures/results/{col.replace("/", "_")}.png')
    plt.show()

# Check normality of residuals

In [ ]:
residuals = sampled_y_pred-sampled_y_true
plt.hist(residuals, bins=100)

In [ ]:
qqplot(residuals, line='s'); plt.show()

In [ ]:
r2_score(scaler.inverse_transform(y_true), scaler.inverse_transform(y_pred), multioutput='raw_values')

In [ ]:
mean_absolute_error(scaler.inverse_transform(y_true),scaler.inverse_transform(y_pred), multioutput='raw_values')